<div style="border: 3px solid #4CAF50; padding: 20px; font-size: 24px; font-weight: bold; font-family: 'Arial', sans-serif; text-align: center; line-height: 1.5; background: linear-gradient(to bottom, #e0f7e9, #ffffff); border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
    פרוייקט כריית וניתוח נתונים בפייתון<br>
    מגישי הפרוייקט : יוסף בנקורמונו - 3184555904<br>
    אריאל קורן - 318284239
</div>


In [1]:
import pandas as pd  # ייבוא pandas לעבודה עם מסגרות נתונים
from datetime import datetime, timedelta
import calendar
import matplotlib.pyplot as plt  # ייבוא matplotlib ליצירת גרפים
import numpy as np  # ייבוא numpy לחישובים מתמטיים
import urllib.request, urllib.parse, urllib.error  # ייבוא ספריות לעבודה עם בקשות רשת
import requests  # ייבוא requests לשליחת בקשות HTTP
from bs4 import BeautifulSoup  # ייבוא BeautifulSoup לניתוח HTML
import re  # ייבוא re לשימוש בביטויים רגולריים

<p style="color:green; font-weight:bold;">שליפת נתונים מהאתר ad.co.il</p>

In [2]:
url = "https://www.ad.co.il/car?sp261=13927"  # כתובת ה-URL של האתר
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}  # הגדרת User-Agent
response = requests.get(url, headers=headers)  # שליחת בקשה לאתר
if response.status_code == 200:
    print("Success")  # הדפסת הודעת הצלחה אם הבקשה הצליחה
else:
    print("Failure")  # הדפסת הודעת כישלון אם הבקשה נכשלה

soup = BeautifulSoup(response.content, 'html.parser')  # ניתוח תוכן ה-HTML
soup

Success


<!DOCTYPE html>

<html data-bs-theme="" dir="rtl" lang="he">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>רכבים פרטיים של יגואר יד שניה למכירה - ad</title>
<meta content="כל הדגמים של יגואר ועוד מגוון רכבים פרטיים נמצאים פה בקליק, הכנסו להתרשמות." name="description" property="og:description">
<link href="https://www.ad.co.il/car?sp261=13927" rel="canonical"/>
<meta content="https://www.ad.co.il/images/logo/ad.png" property="og:image">
<link href="/images/icons/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/images/icons/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="/images/icons/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="/images/icons/apple-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/images/icons/apple-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/>
<link href="/images/icons/apple-icon-120x120.png" rel="appl

<p style="color:green; font-weight:bold;">תרגום מילים מעברית לאנגלית</p>

In [3]:
key_dict = {
    'שנה': 'Year',  # תרגום שנה
    'יד': 'Hand',  # תרגום יד
    'ת. הילוכים': 'Gear',  # תרגום ת. הילוכים
    'נפח': 'Cappacity_engine',  # תרגום נפח
    'סוג מנוע': 'Engine_type',  # תרגום סוג מנוע
    'ק"מ': 'Kilometers',  # תרגום ק"מ
    'טסט עד': 'Test_until',  # תרגום טסט עד
    'צבע': 'Color',  # תרגום צבע
    'בעלות קודמת': 'Prev_ownership',  # תרגום בעלות קודמת
    'בעלות נוכחית': 'Curr_ownership',  # תרגום בעלות נוכחית
    'אזור': 'Area',  # תרגום אזור
    'עיר': 'City'  # תרגום עיר
}

In [4]:
All_Jaguar_ads = []  # רשימה לאחסון כל המודעות של יגואר
translated_car_details = {}  # מילון לאחסון פרטי הרכב המתורגמים
ads_div_tags = soup.find_all('div', class_='card-block')  # מציאת כל תגי ה-div של המודעות

In [5]:
for addiv in ads_div_tags:
    search_ad = addiv.find('a')['href']  # מציאת הקישור למודעה
    ad_url = f'https://www.ad.co.il{search_ad}'  # בניית כתובת ה-URL המלאה למודעה
    Ad_response = requests.get(ad_url, headers=headers)  # שליחת בקשה למודעה
    if Ad_response.status_code == 200:
        print("Success")  # הדפסת הודעת הצלחה אם הבקשה הצליחה
    else:
        print("Failure, the Ad is not available")  # הדפסת הודעת כישלון אם הבקשה נכשלה

    Ad_soup = BeautifulSoup(Ad_response.content, 'html.parser')  # ניתוח תוכן ה-HTML של המודעה
    Ad_details = Ad_soup.find('div', class_='col-xxl-4 col-lg-4 col-md-5')  # מציאת תגי ה-div עם פרטי המודעה
    car_details = {}  # מילון לאחסון פרטי הרכב
    manufact_spans = Ad_soup.find_all('span', itemprop="name")  # מציאת כל תגי ה-span עם שם היצרן
    manufactor = manufact_spans[2].text.strip()  # חילוץ שם היצרן

    price_model_tags = Ad_details.find_all('h2', class_="card-title")  # מציאת תגי ה-h2 עם פרטי המחיר והמודל
    price = price_model_tags[1].get_text()  # חילוץ המחיר
    price = price.replace('₪', '').replace(',', '').strip()  # ניקוי המחיר מסימנים לא רצויים
    model = price_model_tags[0].get_text()  # חילוץ המודל
    model_pallet = re.sub(r'[^a-zA-Z]', '', model)  # ניקוי המודל מסימנים לא רצויים
    model = model_pallet if model_pallet else ""  # שמירת המודל הנקי או מחרוזת ריקה אם אין מודל

    for row in Ad_details.find_all('tr'):
        car_details['Manufactor'] = manufactor  # שמירת שם היצרן
        car_details['Model'] = model  # שמירת שם המודל
        columns = row.find_all('td')  # מציאת כל תגי ה-td בשורה
        if len(columns) == 2:
            label = columns[0].get_text(strip=True)  # חילוץ התווית
            if label in key_dict:
                label = key_dict[label]  # תרגום התווית
                value = columns[1].get_text(strip=True)  # חילוץ הערך
                car_details[label] = value  # שמירת הערך במילון פרטי הרכב

    car_details['Price'] = price  # שמירת המחיר במילון פרטי הרכב

    ad_elem = Ad_soup.find_all('div', class_='px-3')  # מציאת כל תגי ה-div עם תאריך יצירה והקפצה
    for div in ad_elem:
        text = div.text.strip()
        if text.startswith('תאריך יצירה'):
            car_details['Cre_date'] = div.text.split(': ')[1]  # חילוץ ושמירת תאריך יצירה
        elif text.startswith('תאריך הקפצה אחרון'):
            car_details['Repub_date'] = div.text.split(': ')[1]  # חילוץ ושמירת תאריך הקפצה אחרון

    ad_description = Ad_soup.find_all('p', class_='text-word-break')  # מציאת כל תגי ה-p עם תיאור המודעה
    for d in ad_description:
        car_details['Description'] = d.text.strip().replace('\n', '').replace('\r', '').replace('\t', ' ')  # ניקוי ושמירת התיאור

    ad_pictures = Ad_soup.find_all('div', class_="justify-content-center px-1")  # מציאת כל תגי ה-div עם תמונות
    car_details['Pic_num'] = len(ad_pictures)  # שמירת מספר התמונות

    All_Jaguar_ads.append(car_details)  # הוספת פרטי הרכב לרשימה

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


<p style="color:green; font-weight:bold;">יצירת מסגרת נתונים (DataFrame) מהמודעות</p>

In [6]:
Ads_df = pd.DataFrame(All_Jaguar_ads)  # יצירת DataFrame מהמילון
Ads_df = Ads_df[Ads_df['Manufactor'] == 'יגואר']  # סינון המודעות של יגואר בלבד
Ads_df = Ads_df.rename(columns={'Gear': 'Engine_Type'})  # שינוי שם העמודה 'Gear' ל-'Engine_Type'
Ads_df = Ads_df.rename(columns={'Engine_type': 'Gear'})  # שינוי שם העמודה 'Engine_type' ל-'Gear'
Ads_df = Ads_df.rename(columns={'Test_until': 'Test'})

print(Ads_df)

  Manufactor  Model  Year Hand Engine_Type Cappacity_engine        Gear  \
0      יגואר     XK  2007    4    אוטומטית            4,200       בנזין   
1      יגואר     XF  2012    5    אוטומטית            5,000       בנזין   
2      יגואר     XF  2012    3    אוטומטית            3,000        דיזל   
3      יגואר  FType  2018    2    אוטומטית            2,000       בנזין   
4      יגואר     XE  2017    1    אוטומטית            2,000       בנזין   
5      יגואר     XF  2013    5    אוטומטית            2,000       בנזין   
6      יגואר  FType  2017    3    אוטומטית            2,000        דיזל   
7      יגואר    XJS  1990    2    אוטומטית            5,000       בנזין   
8      יגואר    XJS  1990    1    אוטומטית            5,500       בנזין   
9      יגואר    XJR  2021    1    אוטומטית                3  טורבו דיזל   

  Kilometers     Test       Color Prev_ownership Curr_ownership  \
0    176,000  10/2024  כסוף מטאלי          פרטית          פרטית   
1    118,000  10/2024        שחור       


<p style="color:green; font-weight:bold;">השלמת ערכים חסרים והמרת סוגי נתונים</p>

In [7]:
Ads_df['Year'] = Ads_df['Year'].fillna(0).astype(str).str.replace(',', '').astype(int)  # השלמת ערכים חסרים והמרה למספר שלם
Ads_df['Hand'] = Ads_df['Hand'].fillna(-5).astype(str).str.replace(',', '').astype(int)  # השלמת ערכים חסרים והמרה למספר שלם
Ads_df['Gear'] = Ads_df['Gear'].astype('category')  # המרה לקטגוריה
Ads_df['Cappacity_engine'] = Ads_df['Cappacity_engine'].fillna(-5).astype(str).str.replace(',', '').astype(int)  # השלמת ערכים חסרים והמרה למספר שלם
Ads_df['Engine_Type'] = Ads_df['Engine_Type'].astype('category')  # המרה לקטגוריה
Ads_df['Prev_ownership'] = Ads_df['Prev_ownership'].astype('category')  # המרה לקטגוריה
Ads_df['Curr_ownership'] = Ads_df['Curr_ownership'].astype('category')  # המרה לקטגוריה
Ads_df['Area'] = Ads_df['Area'].astype('str')  # המרה למחרוזת
Ads_df['City'] = Ads_df['City'].astype('str')  # המרה למחרוזת
Ads_df['Price'] = Ads_df['Price'].astype(float)  # המרה למספר ממשי

<p style="color:green; font-weight:bold;">מציאת מספר הימים עד לטסט</p>

In [8]:
#    לסטרינג הקיים : הפונקציה מוסיפה את היום האחרון של החודש והופך לתאריך
# הפונקציה ממירה אותו לערך לא הגיוני NaT לערך תאריך לא תקין


def convert_to_last_day_of_month(date_str):
    try:
        if pd.isna(date_str):
            return datetime(year=1800, month=1, day=1)  # אם יש NaN, החזרת תאריך בשנת 1800
        # פיצול המחרוזת לחודש ושנה והמרתם למספרים שלמים
        month, year = map(int, date_str.split('/'))
        # מציאת היום האחרון בחודש
        last_day = calendar.monthrange(year, month)[1]
        # בניית תאריך תקין באמצעות המודול datetime
        return datetime(year=year, month=month, day=last_day)
    except ValueError as e:
        print(f"Error parsing date: {date_str}, {e}")
        return datetime(year=1800, month=1, day=1)  # אם יש שגיאה, החזרת תאריך בשנת 1800

# # החלפת העמודה 'Test' בערכים שהם היום האחרון של החודש
Ads_df['Test'] = Ads_df['Test'].apply(convert_to_last_day_of_month)
print(Ads_df['Test'])

0   2024-10-31
1   2024-10-31
2   1800-01-01
3   1800-01-01
4   1800-01-01
5   1800-01-01
6   1800-01-01
7   1800-01-01
8   1800-01-01
9   1800-01-01
Name: Test, dtype: datetime64[ns]


In [9]:
# מציאת כמות הימים לטסט והפיכה ל int
def days_until_date(date):
    today = pd.Timestamp.now().normalize()  # קביעת התאריך הנוכחי לטיפוס Timestamp
    return (date - today).days

# החלפת הערכים בעמודה 'Test' למספר הימים הנותרים
Ads_df['Test'] = Ads_df['Test'].apply(days_until_date)
print(Ads_df['Test'])
Ads_df['Test'] = Ads_df['Test'].astype(str).astype(int)  # השלמת ערכים חסרים והמרה למספר שלם


0      127
1      127
2   -81991
3   -81991
4   -81991
5   -81991
6   -81991
7   -81991
8   -81991
9   -81991
Name: Test, dtype: int64




<p style="color:green; font-weight:bold;">המרת עמודות תאריך</p>

In [10]:
Ads_df['Cre_date'] = pd.to_datetime(Ads_df['Cre_date'], errors='coerce')  # המרת עמודת תאריך יצירה לתאריך
Ads_df['Repub_date'] = pd.to_datetime(Ads_df['Repub_date'], errors='coerce')  # המרת עמודת תאריך הקפצה לתאריך

C:\Users\ariel\AppData\Local\Temp\ipykernel_10620\2737070140.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  Ads_df['Repub_date'] = pd.to_datetime(Ads_df['Repub_date'], errors='coerce')  # המרת עמודת תאריך הקפצה לתאריך



<p style="color:green; font-weight:bold;">שינוי שם והמרת עמודת 'Kilometers'</p>

In [11]:
Ads_df = Ads_df.rename(columns={'Kilometers': 'KM'})  # שינוי שם העמודה 'Kilometers' ל-'KM'
Ads_df['KM'] = Ads_df['KM'].fillna(-5).astype(str).str.replace(',', '').astype(int)  # השלמת ערכים חסרים והמרה למספר שלם


<p style="color:green; font-weight:bold;">הדפסת המידע של מסגרת הנתונים כדי לוודא את השינויים</p>

In [12]:
print(Ads_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Manufactor        10 non-null     object        
 1   Model             10 non-null     object        
 2   Year              10 non-null     int32         
 3   Hand              10 non-null     int32         
 4   Engine_Type       10 non-null     category      
 5   Cappacity_engine  10 non-null     int32         
 6   Gear              10 non-null     category      
 7   KM                10 non-null     int32         
 8   Test              10 non-null     int32         
 9   Color             5 non-null      object        
 10  Prev_ownership    5 non-null      category      
 11  Curr_ownership    5 non-null      category      
 12  Area              10 non-null     object        
 13  City              10 non-null     object        
 14  Price             10 non-null     


<p style="color:green; font-weight:bold;"> API</p>


<p style="color:green; font-weight:bold;"> ניקוי והתאמת הנתונים מה-API</p>

In [13]:
import pandas as pd
import requests

# פונקציה לשליפת כל הנתונים מה-API
def get_all_vehicle_data():
    url = "https://data.gov.il/api/3/action/datastore_search"
    params = {
        "resource_id": "5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6",
        "limit": 1000  # הגבלת מספר הרשומות בכל עמוד
    }
    
    all_records = []
    offset = 0
    
    while True:
        params["offset"] = offset
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            records = data.get('result', {}).get('records', [])
            
            if not records:
                break  # אין יותר רשומות
            
            all_records.extend(records)
            offset += len(records)  # עדכון ה-offset להבא
        else:
            print(f"Failed to retrieve data: {response.status_code}")
            break
    
    # להמיר את כל הרשומות ל-DataFrame עבור נוחות
    df = pd.DataFrame(all_records)
    return df

# שליפת כל הנתונים מה-API
vehicle_data = get_all_vehicle_data()

# ניקוי והתאמת הנתונים מה-API
vehicle_data = vehicle_data[['tozar', 'kinuy_mishari', 'shnat_yitzur']]
vehicle_data = vehicle_data.rename(columns={
    'tozar': 'Manufactor',
    'kinuy_mishari': 'Model',
    'shnat_yitzur': 'Year'
})

# הסרת נפח מנוע מעמודת 'Model'
def process_model(model):
    parts = model.split()
    base_model = parts[0] if len(parts) > 0 else model
    return base_model

vehicle_data['Model'] = vehicle_data['Model'].apply(lambda x: process_model(x)).str.replace(r'[-\s]', '', regex=True).str.upper()

# המרת העמודה 'Year' ל- str בשני המסגרות
vehicle_data['Year'] = vehicle_data['Year'].astype(str)

# יצירת עמודת 'supply_score' עם ספירת מספר המכוניות לפי יצרן, דגם ושנה
supply_score = vehicle_data.groupby(['Manufactor', 'Model', 'Year']).size().reset_index(name='supply_score')

# התאמת עמודות בדאטה המקורית
Ads_df['Year'] = Ads_df['Year'].astype(str)
Ads_df['Model'] = Ads_df['Model'].str.replace(r'[-\s]', '', regex=True).str.upper()

# מיזוג supply_score עם הדאטה המקורית
Ads_df = pd.merge(Ads_df, supply_score, on=['Manufactor', 'Model', 'Year'], how='left')

# המרת 'supply_score' ל-int ומילוי ערכים חסרים ב-0
Ads_df['supply_score'] = Ads_df['supply_score'].fillna(0).astype(int)

# מיון הדאטה פריים לפי עמודת 'supply_score' בסדר יורד
Ads_df = Ads_df.sort_values(by='supply_score', ascending=False)



<p style="color:green; font-weight:bold;"> Ads_df הדפסה של</p>

In [17]:
Ads_df

,Manufactor,Model,Year,Hand,Engine_Type,Cappacity_engine,Gear,KM,Test,Color,Prev_ownership,Curr_ownership,Area,City,Price,Cre_date,Repub_date,Description,Pic_num,supply_score
4,יגואר,XE,2017,1,אוטומטית,2000,בנזין,156000,-81991,NaN,NaN,NaN,מודיעין והסביבה,מודיעין מכבים רעות,124000.0,NaT,2022-07-15,"פרסטיג' פלוס. יד ראשונה , פרטי. שמור היטבטופל ...",1,21
5,יגואר,XF,2013,5,אוטומטית,2000,בנזין,180,-81991,NaN,NaN,NaN,גליל ועמקים,נצרת,75000.0,2022-01-07,2022-07-01,אוטו במצב חדש לא חסר בורג דגם הכי מפואר מחירון...,1,10
1,יגואר,XF,2012,5,אוטומטית,5000,בנזין,118000,127,שחור,פרטית,פרטית,ירושלים והסביבה,ירושלים,89000.0,2024-05-04,2024-04-06,תוספות קיט XFRSגג נפטח גנט 20׳ ברישוןהמון ה...,10,9
2,יגואר,XF,2012,3,אוטומטית,3000,דיזל,230000,-81991,כסף מטלי,פרטית,פרטית,חיפה וחוף הכרמל,חיפה,82000.0,2022-07-11,2022-11-07,תוספות צבע כסף מט (מדבקה) אפשר להורידצבע מק...,1,9
6,יגואר,FTYPE,2017,3,אוטומטית,2000,דיזל,142000,-81991,NaN,NaN,NaN,גליל ועמקים,קרית שמונה,175000.0,2022-03-05,2022-05-03,יגואר דגם f pace שנת 2017,1,8
3,יגואר,FTYPE,2018,2,אוטומטית,2000,בנזין,85000,-81991,NaN,NaN,NaN,תל אביב,תל אביב יפו,220000.0,2022-04-09,2022-09-04,"רכב מפואר, יוקרתי ומפנק למביני עניין.מצב מעולה...",1,5
0,יגואר,XK,2007,4,אוטומטית,4200,בנזין,176000,127,כסוף מטאלי,פרטית,פרטית,גליל ועמקים,ראש פינה,90000.0,2024-04-06,2024-06-15,תוספות מסך מגע מקורי כניסה והנעה ללא מפתח ג...,5,0
7,יגואר,XJS,1990,2,אוטומטית,5000,בנזין,77000,-81991,שחור,פרטית,אחר,רעננה - כפר סבא,כפר סבא,139000.0,NaT,2022-02-06,"מדובר ביגואר שנת XJSC V-12 1987, ליטר 5.3 , 77...",8,0
8,יגואר,XJS,1990,1,אוטומטית,5500,בנזין,97682,-81991,לבן,פרטית,ייבוא אישי,רעננה - כפר סבא,כפר סבא,119000.0,NaT,2022-02-06,"xjs v12 5.3,,הרכב במצב מעולה, סה""כ נסע 97,682 ...",15,0
9,יגואר,XJR,2021,1,אוטומטית,3,טורבו דיזל,-5,-81991,NaN,NaN,NaN,ירושלים והסביבה,ירושלים,100000.0,NaT,2022-04-21,NaN,0,0



<p style="color:green; font-weight:bold;">ווידוא עמודת supply score בint</p>

In [22]:
print(Ads_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 4 to 9
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Manufactor        10 non-null     object        
 1   Model             10 non-null     object        
 2   Year              10 non-null     object        
 3   Hand              10 non-null     int32         
 4   Engine_Type       10 non-null     category      
 5   Cappacity_engine  10 non-null     int32         
 6   Gear              10 non-null     category      
 7   KM                10 non-null     int32         
 8   Test              10 non-null     int32         
 9   Color             5 non-null      object        
 10  Prev_ownership    5 non-null      category      
 11  Curr_ownership    5 non-null      category      
 12  Area              10 non-null     object        
 13  City              10 non-null     object        
 14  Price             10 non-null     

In [20]:
# ייצוא הדאטה פריים לקובץ CSV
Ads_df.to_csv('Ads_df.csv', index=False, encoding='utf-8-sig', sep=',')

# הצגת קישור להורדת הקובץ ב-Jupyter Notebook
from IPython.display import FileLink
FileLink('Ads_df.csv')

C:\Users\ariel\my notebooks\Ads_df.csv